[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_ml_compare/EMQA_ml_compare.ipynb)

# EMQA_ml_compare

Rolling 1-step-ahead ML model comparison: Linear Regression, Random Forest, Gradient Boosting on Brent crude oil returns.
Evaluates using **R²_OOS** (vs naive benchmark), Direction Accuracy, and Sharpe Ratio.

**Key Finding:** ML models show marginal improvement over naive for oil prices.

**Output:** `ml_compare.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()

brent = fetch('BZ=F', start='2018-01-01')
df = pd.DataFrame({'price': brent})
df['return'] = np.log(df['price'] / df['price'].shift(1))

# Features: lag1-lag5 returns, rolling vol and mean (5,20)
for lag in range(1, 6):
    df[f'ret_lag_{lag}'] = df['return'].shift(lag)

df['roll_vol_5']  = df['return'].rolling(5).std()
df['roll_vol_20'] = df['return'].rolling(20).std()
df['roll_mean_5']  = df['return'].rolling(5).mean()
df['roll_mean_20'] = df['return'].rolling(20).mean()

# Target: next-day return
df['target'] = df['return'].shift(-1)
df = df.dropna()

feature_cols = [c for c in df.columns if c not in ['price', 'return', 'target']]
print(f"Features: {feature_cols}")
print(f"Total observations: {len(df)}")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Rolling expanding-window 1-step-ahead forecast for 3 models
init_train = int(len(df) * 0.7)
retrain_every = 20

model_specs = {
    'Linear Regression': None,
    'Random Forest': None,
    'Gradient Boosting': None,
}

# Storage
preds = {m: [] for m in model_specs}
actuals = []
dates = []
yesterday_prices = []

for i in range(init_train, len(df)):
    step = i - init_train

    # Retrain every 20 steps or first iteration
    if step % retrain_every == 0:
        X_tr = df[feature_cols].iloc[:i].values
        y_tr = df['target'].iloc[:i].values

        model_specs['Linear Regression'] = LinearRegression().fit(X_tr, y_tr)
        model_specs['Random Forest'] = RandomForestRegressor(
            n_estimators=200, max_depth=10, random_state=42, n_jobs=-1).fit(X_tr, y_tr)
        model_specs['Gradient Boosting'] = GradientBoostingRegressor(
            n_estimators=200, max_depth=5, random_state=42).fit(X_tr, y_tr)

    X_step = df[feature_cols].iloc[i:i+1].values

    for name, model in model_specs.items():
        preds[name].append(model.predict(X_step)[0])

    actuals.append(df['target'].iloc[i])
    yesterday_prices.append(df['price'].iloc[i])
    dates.append(df.index[i])

actuals = np.array(actuals)
dates = pd.DatetimeIndex(dates)
yesterday_prices = np.array(yesterday_prices)

for m in preds:
    preds[m] = np.array(preds[m])

# --- Naive benchmark for returns: zero (no change) ---
naive_preds = np.zeros_like(actuals)

mse_naive = mean_squared_error(actuals, naive_preds)

# Metrics
results = {}
print("=" * 70)
print("   ML Model Comparison vs Naive (return = 0)")
print("=" * 70)
print(f"{'Model':<22} {'R²_OOS':>10} {'Direction':>10} {'Sharpe':>10}")
print("-" * 70)

for name in model_specs:
    y_pred = preds[name]
    mae = mean_absolute_error(actuals, y_pred)
    mse_model = mean_squared_error(actuals, y_pred)
    
    # R²_OOS = 1 - MSE_model / MSE_naive
    r2_oos = 1 - mse_model / mse_naive

    # Direction accuracy
    actual_dir = np.sign(actuals)
    pred_dir = np.sign(y_pred)
    dir_acc = np.mean(actual_dir == pred_dir) * 100

    # Directional Sharpe: trade sign of predicted return
    strat_ret = np.sign(y_pred) * actuals
    sharpe = strat_ret.mean() / strat_ret.std() * np.sqrt(252) if strat_ret.std() > 0 else 0

    results[name] = {'MAE': mae, 'R2_OOS': r2_oos, 'Direction': dir_acc, 'Sharpe': sharpe}
    print(f"{name:<22} {r2_oos*100:>9.1f}% {dir_acc:>9.1f}% {sharpe:>10.2f}")

print("=" * 70)

res_df = pd.DataFrame(results).T

In [ ]:
# Plot 1: Cumulative returns of each model's strategy vs buy-and-hold
fig, ax = plt.subplots(figsize=(12, 6))

# Buy-and-hold cumulative return
cum_bh = np.cumsum(actuals)
ax.plot(dates, cum_bh, color=COLORS['gray'], lw=1.8, label='Buy & Hold')

model_colors = [COLORS['blue'], COLORS['green'], COLORS['orange']]
for (name, y_pred), col in zip(preds.items(), model_colors):
    strat_ret = np.sign(y_pred) * actuals
    cum_strat = np.cumsum(strat_ret)
    ax.plot(dates, cum_strat, color=col, lw=1.5, label=name)

ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Log Return')
ax.set_title('Rolling 1-Step-Ahead: Cumulative Strategy Returns vs Buy & Hold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=4)

plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Bar chart comparing R²_OOS, Direction, Sharpe for each model
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

bar_colors = [COLORS['blue'], COLORS['green'], COLORS['orange']]
metrics = ['R2_OOS', 'Direction', 'Sharpe']
titles = ['R²_OOS vs Naive (higher is better)',
          'Direction Accuracy % (higher is better)',
          'Sharpe Ratio (higher is better)']
ylabels = ['R²_OOS', 'Direction %', 'Sharpe']
formats = ['{:.1%}', '{:.1f}%', '{:.2f}']

for ax, metric, title, ylabel, fmt in zip(axes, metrics, titles, ylabels, formats):
    vals = res_df[metric].values
    bars = ax.bar(res_df.index, vals, color=bar_colors, width=0.5, edgecolor='white')
    for bar, val in zip(bars, vals):
        offset = abs(val) * 0.03 if val != 0 else 0.001
        va = 'bottom' if val >= 0 else 'top'
        if metric == 'R2_OOS':
            label = f'{val*100:.1f}%'
        elif metric == 'Direction':
            label = f'{val:.1f}%'
        else:
            label = f'{val:.2f}'
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + offset,
                label, ha='center', va=va, fontsize=10)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=15)
    if metric in ['R2_OOS']:
        ax.axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
    if metric == 'Direction':
        ax.axhline(50, color=COLORS['red'], linestyle='--', linewidth=1, label='Coin flip (50%)')
        ax.legend(loc='lower right', fontsize=9)

plt.tight_layout()
save_fig(fig, 'ml_compare.pdf')
plt.show()